In [1]:
import torch
torch.cuda.empty_cache()

In [2]:
# Ensures that there is enough memory allocation for the model to load
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

In [4]:
from groundingdino.util.inference import load_model, load_image, predict, annotate
import cv2

model = load_model("/home/jovyan/GroundingDINO/groundingdino/config/GroundingDINO_SwinT_OGC.py", "/home/jovyan/GroundingDINO/weights/groundingdino_swint_ogc.pth")

final text_encoder_type: bert-base-uncased


In [5]:
import json
def save_labels(labels_list, objects,image_path,precision):
    incorrect = [item for item in labels_list if item not in objects]
    data = {
        "Image Path" : image_path,
        "Original Labels" : labels_list, 
        "Correct Labels" :  objects,
        "Incorrect Labels" : incorrect,
        "Precision" : precision
     }
    
    new_path = '/home/jovyan/Evaluation/Labels'
    
    if not os.path.exists(new_path):
        os.makedirs(new_path) # Create the directory if it doesn't exist
        print(f"{new_path} successfully created")
    name = image_path.strip(".jpg").strip(".png")
    json_filename = f'{name}.json'
    file_name = os.path.join(new_path , json_filename)
    with open(file_name , 'w') as json_file:
            json.dump(data, json_file, indent=4)
    print(f"File : {json_filename} successfully created!")

In [6]:
def load_files(file):
    # Open and read the JSON file
    path = os.path.join('/home/jovyan/Evaluation/Data(Gemma)', file)
    with open(path, 'r') as json_file:
        data = json.load(json_file)
    return data

In [7]:

# Initiate variables 
success = 0
# Path to the directory (library)
directory_path = "/home/jovyan/Evaluation/Data(Gemma)"  # Replace this with the actual path

# List all files in the directory
files = os.listdir(directory_path)

# Filter out directories and show only files
files = [file for file in files if os.path.isfile(os.path.join(directory_path, file))]

# Print the list of files
for index, file in enumerate(files):
    torch.cuda.empty_cache()
    image_path = file.strip(".json")
    print(f"Processing Image {image_path}")
    original_image = os.path.join('/home/jovyan/images',image_path)
    
    data = load_files(file)
    
    labels_list = data['Labels']
    lists = set(labels_list)
    correct = 0
    objects = []
    # Takes in the prompt in this format , "cat. dog. person."
    for label in lists:
        TEXT_PROMPT = f"{label}."

        BOX_TRESHOLD = 0.25
        TEXT_TRESHOLD = 0.25

        image_source, image = load_image(original_image)

        boxes, logits, phrases = predict(
            model=model,
            image=image,
            caption=TEXT_PROMPT,
            box_threshold=BOX_TRESHOLD,
            text_threshold=TEXT_TRESHOLD
        )
        if label in phrases:
            correct += 1 
            objects.append(label)
    success += correct/len(labels_list) * 100
    precision = round(correct/len(lists),3)
    save_labels(list(lists),objects,image_path,precision)
    print(f"Successfully processed Image {file}, {index+1}/{len(files)} Images Processed")
rate = (success / len(files))
print(f"Model has a success rate of {round(rate,1)}% over {index + 1} images.")

Processing Image 506.jpg


/home/jovyan/Evaluation/Labels successfully created
File : 506.json successfully created!
Successfully processed Image 506.jpg.json, 1/265 Images Processed
Processing Image 763.jpg
File : 763.json successfully created!
Successfully processed Image 763.jpg.json, 2/265 Images Processed
Processing Image 660.jpg
File : 660.json successfully created!
Successfully processed Image 660.jpg.json, 3/265 Images Processed
Processing Image 738.jpg
File : 738.json successfully created!
Successfully processed Image 738.jpg.json, 4/265 Images Processed
Processing Image 579.jpg
File : 579.json successfully created!
Successfully processed Image 579.jpg.json, 5/265 Images Processed
Processing Image 560.jpg
File : 560.json successfully created!
Successfully processed Image 560.jpg.json, 6/265 Images Processed
Processing Image 708.png
File : 708.json successfully created!
Successfully processed Image 708.png.json, 7/265 Images Processed
Processing Image 630.jpg
File : 630.json successfully created!
Success